In [19]:
import pandas as pd
import numpy as np
import sklearn
import random
from sklearn import datasets
import seaborn.apionly as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [84]:
titanic = sns.load_dataset('titanic')
del titanic['alive']
del titanic['embarked']

In [85]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,Third,man,True,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,First,woman,False,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,Third,woman,False,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,First,woman,False,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,Third,man,True,NaN,Southampton,True


Transform the categoricals to a label based system

In [86]:
cat_cols = ['sex', 'class', 'who', 'deck', 'embark_town']

In [87]:
for c in cat_cols:
    label = LabelEncoder()
    titanic[c] = label.fit_transform(titanic[c])

In [88]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,2,1,True,0,3,False
1,1,1,0,38.0,1,0,71.2833,0,2,False,3,1,False
2,1,3,0,26.0,0,0,7.9250,2,2,False,0,3,True
3,1,1,0,35.0,1,0,53.1000,0,2,False,3,3,False
4,0,3,1,35.0,0,0,8.0500,2,1,True,0,3,True


check for NA values and replace them with the median if numerical

In [89]:
titanic.isnull().any()

survived       False
pclass         False
sex            False
age             True
sibsp          False
parch          False
fare           False
class          False
who            False
adult_male     False
deck           False
embark_town    False
alone          False
dtype: bool

In [90]:
titanic['age'].fillna(titanic['age'].median(), inplace = True)

In [91]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,2,1,True,0,3,False
1,1,1,0,38.0,1,0,71.2833,0,2,False,3,1,False
2,1,3,0,26.0,0,0,7.9250,2,2,False,0,3,True
3,1,1,0,35.0,1,0,53.1000,0,2,False,3,3,False
4,0,3,1,35.0,0,0,8.0500,2,1,True,0,3,True


Split train and test data

In [92]:
Y = titanic['survived']
X = titanic.iloc[:,1:]
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.2, random_state = 666)

## Random Forest

In [106]:
rf = RandomForestClassifier(n_estimators=10000)
rf.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [107]:
fpred = rf.predict(xtest)
accuracy_score(fpred, ytest)

0.82681564245810057

## Gradient Boosting Classifier

In [104]:
gb = GradientBoostingClassifier(n_estimators=1000, max_depth = 25, learning_rate=0.001)
gb.fit(xtrain, ytrain)

GradientBoostingClassifier(init=None, learning_rate=0.001, loss='deviance',
              max_depth=25, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [105]:
gpred = gb.predict(xtest)
accuracy_score(gpred, ytest)

0.8044692737430168